Description: Este código tem como objetivo extrair os discursos dos 3 últimos presidentes (Temer, Dilma e Lula) do site do Itamaraty. 

Created by: Leandro A. da Silva  (leandro@kognita.com.br)

Created at: 20/6/2017

Last modification: 18/6/2018

<< Código para fins didáticos, apenas >>

#### Matéria do El Pais (15/06/18): https://brasil.elpais.com/brasil/2018/06/15/politica/1529014069_205306.html

"Governo altera sistema de busca em site do Planalto e dificulta pesquisa em discursos de Temer"

#### Carregando as bibliotecas necessárias

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk import bigrams
from itertools import chain
from datetime import datetime
from unidecode import unidecode
import time

"Request" in order to collect all pages that contains speeches:

#### O objetivo é coletar os discursos contidos na seguinte página:

In [3]:
url = 'http://www.itamaraty.gov.br/pt-BR/discursos-artigos-e-entrevistas-categoria/presidente-da-republica-federativa-do-brasil-discursos'

In [4]:
print url

http://www.itamaraty.gov.br/pt-BR/discursos-artigos-e-entrevistas-categoria/presidente-da-republica-federativa-do-brasil-discursos


#### O endereço base para o scraping:

In [5]:
url_base = 'http://www.itamaraty.gov.br/pt-BR/discursos-artigos-e-entrevistas-categoria/presidente-da-republica-federativa-do-brasil-discursos?start='

In [6]:
print url_base

http://www.itamaraty.gov.br/pt-BR/discursos-artigos-e-entrevistas-categoria/presidente-da-republica-federativa-do-brasil-discursos?start=


#### Fazendo um loop sobre as páginas (variando de 0 a 375 de 15 em 15, conforme inspecionado no site) e armazendo-as numa lista

In [7]:
pages_list = []
for i in xrange(0,375,15):
    time.sleep(1)
    #url base + a string que define o número da página
    one_page = requests.get(str(url_base + str(i)))
    pages_list.append(one_page)

total number of pages:

In [8]:
print len(pages_list)

25



#### Extraíndo o conteúdo html das páginas acima coletadas convertendo-as para um objeto "Beautiful Soup":

In [9]:
soup_list = []
for j in xrange(0,len(pages_list)):
    soup_list.append(BeautifulSoup(pages_list[j].content.decode('utf-8'), 'html.parser'))

In [10]:
print len(soup_list)

25


#### Olhando o aspecto de um dos objetos anexados na lista:

In [11]:
print soup_list[0]

<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="pt-br" dir="ltr"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8" lang="pt-br" dir="ltr"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9" lang="pt-br" dir="ltr"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" dir="ltr" lang="pt-br"> <!--<![endif]-->
<html dir="ltr" lang="pt-br">
<head>
<meta content="f9cA9fEKYT9iDe-lZ6CyyGs0vw5JcBqBhWhgowrSO3c" name="google-site-verification"/>
<!--[if lt IE 9]>
    <script src="/templates/mre/js/html5shiv.js"></script>
    <![endif]-->
<link href="/templates/mre/bootstrap/css/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="/templates/mre/css/template-azul.css" rel="stylesheet" type="text/css"/>
<link href="/templates/mre/css/icones-bmp-azul.css" rel="stylesheet" type="text/css"/>
<link href="/templates/mre/font-awesome/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<link href="/templates/m

#### Extraíndo todos os links das páginas:

In [13]:
links=[]
for j in xrange(0,len(soup_list)):
    for link in soup_list[j].find_all('a'):
        links.append(link.get('href'))

In [16]:
links

[u'#accessibility',
 u'http://brasil.gov.br',
 u'#content',
 u'#navigation',
 u'#portal-searchbox',
 u'#footer',
 u'/pt-BR/acessibilidade',
 u'#',
 u'/pt-BR/?Itemid=142',
 u'/es/discursos-articulos-y-entrevistas/presidente-de-la-republica-federativa-de-brasil-discursos',
 u'/en/speeches-articles-and-interviews/president-of-the-federative-republic-of-brazil-speeches',
 u'http://www.itamaraty.gov.br/',
 u'http://www.youtube.com/mrebrasil',
 u'https://www.facebook.com/ItamaratyGovBr',
 u'https://twitter.com/ItamaratyGovBr',
 u'https://twitter.com/itamaraty_en',
 u'https://twitter.com/itamaraty_es',
 u'https://soundcloud.com/ItamaratyGovBr',
 u'http://www.flickr.com/photos/mrebrasil/',
 u'http://instagram.com/itamaratygovbr',
 u'/pt-BR/midias-digitais',
 u'/pt-BR/perguntas-frequentes',
 u'/pt-BR/servicos',
 u'/pt-BR/dados-abertos',
 u'/pt-BR/fale-conosco',
 u'http://www.portalconsular.mre.gov.br/',
 u'/pt-BR/agendas',
 u'/pt-BR/area-de-imprensa',
 u'#',
 u'/pt-BR/servico-consular',
 u'http

In [12]:
print len(links)
print links[-2:]
print links[-1:]

4172
[u'http://www.brasil.gov.br/', u'#portal-siteactions']
[u'#portal-siteactions']


In [13]:
type(links[0])

unicode


#### Convertendo os links de unicode para string:

In [18]:
linkss = []
for j in xrange(0,len(links)):
    linkss.append(str(links[j]))

In [19]:
len(linkss)

4172

In [20]:
linkss[0]

'#accessibility'

In [16]:
type(linkss[0])

str

#### Por inspeção no site, sabemos que os links relativos aos discursos de cada presidente contém seus respectivos nomes. Assim, podemos separar os links da seguinte forma:

In [23]:
temer = []
dilma = []
lula = []
for i in xrange(0,len(linkss)):
    
    if 'temer' in linkss[i]:
        temer.append(str('http://www.itamaraty.gov.br'+linkss[i]))
    elif 'dilma' in linkss[i]:
         dilma.append(str('http://www.itamaraty.gov.br'+linkss[i]))
    elif 'lula' in linkss[i]:
         lula.append(str('http://www.itamaraty.gov.br'+linkss[i]))


In [24]:
len_temer = len(temer)
len_dilma = len(dilma)
len_lula = len(lula)

print 'discursos Temer: ', len_temer
print 'discursos Dilma: ',len_dilma
print 'discursos Lula: ', len_lula


discursos Temer:  77
discursos Dilma:  159
discursos Lula:  110


#### Verificando alguns links:

In [19]:
print temer[0]

http://www.itamaraty.gov.br/pt-BR/discursos-artigos-e-entrevistas-categoria/presidente-da-republica-federativa-do-brasil-discursos/19047-discurso-do-presidente-da-republica-michel-temer-durante-a-sessao-plenaria-dos-presidentes-dos-estados-partes-do-mercosul-estados-associados-e-convidados-especiais-assuncao-paraguai-18-de-junho-de-2018


In [20]:
print dilma[0]

http://www.itamaraty.gov.br/pt-BR/discursos-artigos-e-entrevistas-categoria/presidente-da-republica-federativa-do-brasil-discursos/13993-discurso-da-presidenta-da-republica-dilma-rousseff-durante-cerimonia-da-uniao-das-americas-por-ocasiao-da-chegada-da-chama-olimpica-rio-2016-em-brasilia-brasilia-03-de-maio-de-2016


In [21]:
print lula[0]

http://www.itamaraty.gov.br/pt-BR/discursos-artigos-e-entrevistas-categoria/presidente-da-republica-federativa-do-brasil-discursos/5932-discurso-do-presidente-da-republica-luiz-inacio-lula-da-silva-durante-cerimonia-de-posse-solene-do-conselho-de-representantes-brasileiros-no-exterior-no-ambito-da-3-conferencia-brasileiros-no-mundo-rio-de-janeiro-03-de-dezembro-de-2010


#### Extraíndo  o conteúdo html relativo a cada presidente:

In [25]:
temer_pages = []
soup_temer = []
for j in xrange(0,len_temer):
    temer_pages.append(requests.get(temer[j]))
    soup_temer.append(BeautifulSoup(temer_pages[j].content, 'html.parser'))
    

dilma_pages = []
soup_dilma = []
for j in xrange(0,len_dilma):
    dilma_pages.append(requests.get(dilma[j]))
    soup_dilma.append(BeautifulSoup(dilma_pages[j].content, 'html.parser')) 
    
    
lula_pages = []
soup_lula = []
for j in xrange(0,len_lula):
    lula_pages.append(requests.get(lula[j]))
    soup_lula.append(BeautifulSoup(lula_pages[j].content, 'html.parser'))    
    

    

No handlers could be found for logger "bs4.dammit"


In [26]:
print len(temer_pages)
print len(soup_temer)

77
77


In [24]:
print len(dilma_pages)
print len(soup_dilma)

159
159


In [25]:
print len(lula_pages)
print len(soup_lula)

110
110


#### Extraíndo o texto de cada página:

In [27]:
text_temer = []
text_temer_aux = []
for k in xrange(0,len(soup_temer)):
    for j in xrange(0,len(soup_temer[k].find_all('p'))-2):
        text_temer_aux.append(soup_temer[k].find_all('p')[j].get_text())
    text_temer.append(text_temer_aux)
    text_temer_aux = []
    
    
    
text_dilma = []
text_dilma_aux = []
for k in xrange(0,len(soup_dilma)):
    for j in xrange(0,len(soup_dilma[k].find_all('p'))-2):
        text_dilma_aux.append(soup_dilma[k].find_all('p')[j].get_text())
    text_dilma.append(text_dilma_aux)
    text_dilma_aux = []   
    
text_lula = []
text_lula_aux = []
for k in xrange(0,len(soup_lula)):
    for j in xrange(0,len(soup_lula[k].find_all('p'))-2):
        text_lula_aux.append(soup_lula[k].find_all('p')[j].get_text())
    text_lula.append(text_lula_aux)
    text_lula_aux = []    
    

In [28]:
print len(text_temer)

77


In [29]:
print len(text_dilma)

159


In [29]:
print len(text_lula)

110


#### Extraíndo o título de cada discurso:

In [30]:
title_temer = []
title_temer_aux = []
for k in xrange(0,len(soup_temer)):
    for j in xrange(0,len(soup_temer[k].find_all('title'))):
        title_temer_aux.append(soup_temer[k].find_all('title')[j].get_text())
    title_temer.append(title_temer_aux)
    title_temer_aux = []
    
title_dilma = []
title_dilma_aux = []
for k in xrange(0,len(soup_dilma)):
    for j in xrange(0,len(soup_dilma[k].find_all('title'))):
        title_dilma_aux.append(soup_dilma[k].find_all('title')[j].get_text())
    title_dilma.append(title_dilma_aux)
    title_dilma_aux = []  
    
title_lula = []
title_lula_aux = []
for k in xrange(0,len(soup_lula)):
    for j in xrange(0,len(soup_lula[k].find_all('title'))):
        title_lula_aux.append(soup_lula[k].find_all('title')[j].get_text())
    title_lula.append(title_lula_aux)
    title_lula_aux = []      
    
ftitle_temer=list(chain.from_iterable(title_temer))
print len(ftitle_temer)

ftitle_dilma=list(chain.from_iterable(title_dilma))
print len(ftitle_dilma)

ftitle_lula=list(chain.from_iterable(title_lula))
print len(ftitle_lula)

77
159
110


#### Inspecionando alguns títulos:

In [31]:
print ftitle_temer[0]

Discurso do Presidente da República, Michel Temer, durante a Sessão Plenária dos Presidentes dos Estados Partes do Mercosul, Estados Associados e Convidados Especiais - Assunção, Paraguai, 18 de junho de 2018


In [32]:
print ftitle_dilma[0]

Discurso da Presidenta da República, Dilma Rousseff, durante cerimônia da União das Américas por ocasião da chegada da Chama Olímpica Rio 2016 em Brasília – Brasília, 03 de maio de 2016


In [33]:
print ftitle_lula[0]

Discurso do Presidente da República, Luiz Inácio Lula da Silva, durante cerimônia de posse solene do Conselho de Representantes Brasileiros no Exterior, no âmbito da 3ª Conferência "Brasileiros no Mundo" - Rio de Janeiro, 3 de dezembro de 2010


#### Cada um dos textos dos discursos são listas de parágrafos. Observe: 

In [31]:
text_temer[0]

[u'Prezados amigos,',
 u'Presidentes, vice-presidentes,',
 u'Hor\xe1cio Cartes,',
 u'Tabar\xe9 V\xe1zquez,',
 u'Marta Gabriela Michetti,',
 u'\xc1lvaro Garcia Limeira,',
 u'Senhoras e senhores membros das delega\xe7\xf5es,',
 u'Minhas senhoras e meus senhores,',
 u'Eu, naturalmente, come\xe7o por um dever de natureza pol\xedtica e social, que \xe9 cumprimentado o presidente Cartes pelo \xeaxito da sua condu\xe7\xe3o do Mercosul ao longo deste semestre. Registro que a presid\xeancia paraguaia guiou o nosso processo de integra\xe7\xe3o com seguran\xe7a e determina\xe7\xe3o. Hoje \xe9 cada vez mais n\xf3s podemos celebrar um Mercosul ainda mais revigorado, um Mercosul efetivamente a servi\xe7o dos nossos povos. Portanto, eu quero tamb\xe9m, presidente Cartes, felicit\xe1-lo pela organiza\xe7\xe3o desta c\xfapula. N\xe3o \xe9 a primeira vez que, naturalmente, nos encontramos, n\xe3o \xe9 a primeira vez que venho ao Paraguai, mas n\xf3s, sempre que aqui chegamos, nos surpreendemos positivam

#### O seguinte comando pode fundir a lista de parágrafos em único bloco de texto:

In [32]:
print ' '.join(text_temer[0])

Prezados amigos, Presidentes, vice-presidentes, Horácio Cartes, Tabaré Vázquez, Marta Gabriela Michetti, Álvaro Garcia Limeira, Senhoras e senhores membros das delegações, Minhas senhoras e meus senhores, Eu, naturalmente, começo por um dever de natureza política e social, que é cumprimentado o presidente Cartes pelo êxito da sua condução do Mercosul ao longo deste semestre. Registro que a presidência paraguaia guiou o nosso processo de integração com segurança e determinação. Hoje é cada vez mais nós podemos celebrar um Mercosul ainda mais revigorado, um Mercosul efetivamente a serviço dos nossos povos. Portanto, eu quero também, presidente Cartes, felicitá-lo pela organização desta cúpula. Não é a primeira vez que, naturalmente, nos encontramos, não é a primeira vez que venho ao Paraguai, mas nós, sempre que aqui chegamos, nos surpreendemos positivamente pela excepcional hospitalidade em que todos somos recebidos em Assunção. E nós temos, na verdade, motivos de sobra para nos orgulha

#### Generalizando:

In [33]:
ftext_temer = []
for k in xrange(0,len(text_temer)):
    ftext_temer.append(''.join(text_temer[k]))
    
ftext_dilma = []
for k in xrange(0,len(text_dilma)):
    ftext_dilma.append(''.join(text_dilma[k]))    
    
ftext_lula = []
for k in xrange(0,len(text_lula)):
    ftext_lula.append(''.join(text_lula[k]))        

In [34]:
print len(ftext_temer)
print len(ftext_dilma)
print len(ftext_lula)

77
159
110


#### Criando um dataframe para cada presidente:

In [35]:
df_temer = pd.DataFrame()
df_dilma = pd.DataFrame()
df_lula = pd.DataFrame()

#### Definindo o nome das colunas:

In [36]:
df_temer['title'] = ftitle_temer
df_temer['text'] = ftext_temer
df_temer['president'] = 'Temer'

df_dilma['title'] = ftitle_dilma
df_dilma['text'] = ftext_dilma
df_dilma['president'] = 'Dilma'

df_lula['title'] = ftitle_lula
df_lula['text'] = ftext_lula
df_lula['president'] = 'Lula'

#### Vamos inspecionar cada um dos dataframes:

In [37]:
df_temer

,title,text,president
0,"Discurso do Presidente da República, Michel Te...","Prezados amigos,Presidentes, vice-presidentes,...",Temer
1,"Brinde do presidente da República, Michel Teme...","Excelentíssimo senhor Desiré Bouterse, preside...",Temer
2,"Discurso do presidente da República, Michel Te...","Eu quero cumprimentar, em primeiro lugar, a se...",Temer
3,"Discurso do Presidente da República, Michel Te...","Senhor Presidente Vizcarra, senhoras e senhore...",Temer
4,"Discurso do Presidente da República, Michel Te...","Senhoras e senhores chefes de Estado, de gover...",Temer
5,"Discurso do Presidente da República, Michel Te...",Quero cumprimentar senhoras e senhores chefe d...,Temer
6,"Discurso do Presidente da República, Michel Te...","Olha, eu quero começar cumprimentando a embaix...",Temer
7,"Discurso do Presidente da República, Michel Te...",Eu quero cumprimentar o professor Klaus Schwab...,Temer
8,"Discurso do Presidente da República, Michel Te...",Obrigado. Eu agradeço ao presidente Arlindo Ch...,Temer
9,"Brinde do Presidente da República, Michel Teme...","Bem, eu quero saudar, saudar o presidente da R...",Temer


In [41]:
df_dilma

,title,text,president
0,"Discurso da Presidenta da República, Dilma Rou...",Foto: Roberto Stuckert Filho/PRSenhor Carlos A...,Dilma
1,"Discurso da Presidenta da República, Dilma Rou...",Fotos: Ichiro Guerra/PRSenhor Secretário-Geral...,Dilma
2,"Discurso da Presidenta da República, Dilma Rou...",Fotos: Ichiro Guerra/PRSenhor Secretário-Geral...,Dilma
3,"Discurso da Presidenta da República, Dilma Rou...","Boa tarde a todos,Eu queria dirigir um cumprim...",Dilma
4,"Pronunciamento da Presidenta da República, Dil...","Brasileiras e brasileiros,Peço licença para en...",Dilma
5,"Discurso da Presidenta da República, Dilma Rou...","Excelentíssimo senhor Evo Morales, presidente ...",Dilma
6,Declaração à imprensa da Presidenta da Repúbli...,Foto: Roberto Stuckert Filho/PR Excelentíssimo...,Dilma
7,Declaração à imprensa da Presidenta da Repúbli...,Foto: Roberto Stuckert Filho/PR Excelentíssimo...,Dilma
8,"Discurso da Presidenta da República, Dilma Rou...","Excelentíssimo senhor Rosen Plevneliev, presid...",Dilma
9,Declaração à imprensa da Presidenta da Repúbli...,Foto: Roberto Stuckert/PR Excelentíssimo senho...,Dilma


In [42]:
df_lula

,title,text,president
0,"Discurso do Presidente da República, Luiz Inác...",,Lula
1,"Discurso do Presidente da República, Luiz Inác...",,Lula
2,"Mensagem do Presidente da República, Luiz Inác...",,Lula
3,"Discurso do Presidente da República, Luiz Inác...","Excelentíssimo senhor Juan Manuel Santos, pres...",Lula
4,"Discurso do Presidente da República, Luiz Iná...","Excelentíssimo senhor Malam Bacai Sanhá, presi...",Lula
5,"Discurso do Presidente da República, Luiz Inác...","Se o Vinicius de Moraes estivesse aqui, ele ia...",Lula
6,"Discurso do Presidente da República, Luiz Iná...","Meu caro companheiro e amigo Mauricio Funes, p...",Lula
7,"Intervenção do Presidente da República, Luiz I...","Apenas, apenas para dizer aos companheiros Jos...",Lula
8,"Discurso do Presidente da República, Luiz Iná...","Bem, companheiro Chávez,Companheiros empresári...",Lula
9,"Discurso do Presidente da República, Luiz Inác...","Bem, primeiro, cumprimentar o nosso querido co...",Lula


#### Repare que nem todos os textos foram capturados, pois existem linhas na coluna 'text' em branco. Essas entradas teriam que ser analisadas caso a caso para capturarmos as nuances da codificação em html de cada página. Para nossos objetivos, não há problemas. Descartaremos as linhas em branco mais para frente. 

#### As datas exatas dos discursos estão presentes apenas no texto. No campo de data dos artigos, está apenas a data de publicação dos respectivos discursos no site. Portanto, devemos coletar as datas em que os discursos foram feitos diretamente do corpo do título.

#### Para isso:
Let us first define the months of the year in portuguese, and a dictionary that translate months in portuguese into months in english in the pandas format:

In [38]:
months = ['janeiro', 'fevereiro', 'marco', 'abril', 'maio', 'junho', 'julho',
          'agosto', 'setembro', 'outubro', 'novembro','dezembro']

In [39]:
months_dic = {'janeiro':'Jan', 'fevereiro':'Feb', 'marco':'Mar', 'abril':'Apr', 'maio':'May',
              'junho':'Jun','julho':'Jul','agosto':'Aug','setembro':'Sep','outubro':'Oct','novembro':'Nov',
              'dezembro':'Dec'}

#### Procedimento:

1) Use "unidecode" command to eliminate the portuguese accents.

2) Append the unidecoded titles

3) tokenize each title

4) try to find a month in the title

5) using the date pattern, for example: "12 de janeiro de 2015"find the day and the year in the list [Jan, 12, 2015]

6) Perform the "join" command in the list

7) Convert it to pandas datetime object: .date() keeps only the date and drops the hour.

#### Seguindo as instruções para o Temer:

In [43]:
all_titles = []
all_dates = []
for k in xrange(0,len(ftitle_temer[0:5])):
    print k
    all_titles.append(unidecode(ftitle_temer[k]))
    tokens = word_tokenize(all_titles[k])
    print tokens
    match = next((x for x in months if x in tokens), 'False')
    print match
    
    if match != 'False':
        test_index = tokens.index(match)
        print test_index
        date_list = [months_dic[match], tokens[test_index-2].replace('o', ''),tokens[test_index+2]]
        fdate = str(' '.join(date_list))
        datetime_object = datetime.strptime(fdate, '%b %d %Y').date()        

        all_dates.append(datetime_object)
    else:
        all_dates.append('not applicable')
     
          
    

0
['Discurso', 'do', 'Presidente', 'da', 'Republica', ',', 'Michel', 'Temer', ',', 'durante', 'a', 'Sessao', 'Plenaria', 'dos', 'Presidentes', 'dos', 'Estados', 'Partes', 'do', 'Mercosul', ',', 'Estados', 'Associados', 'e', 'Convidados', 'Especiais', '-', 'Assuncao', ',', 'Paraguai', ',', '18', 'de', 'junho', 'de', '2018']
junho
33
1
['Brinde', 'do', 'presidente', 'da', 'Republica', ',', 'Michel', 'Temer', ',', 'durante', 'almoco', 'em', 'homenagem', 'ao', 'senhor', 'Desire', 'Delan', 'Bouterse', ',', 'Presidente', 'da', 'Republica', 'do', 'Suriname', '-', 'Palacio', 'Itamaraty', ',', '2', 'de', 'maio', 'de', '2018']
maio
30
2
['Discurso', 'do', 'presidente', 'da', 'Republica', ',', 'Michel', 'Temer', ',', 'proferido', 'durante', 'a', 'formatura', 'da', 'Turma', 'Marielle', 'Franco', '(', '2016-2018', ')', ',', 'por', 'ocasiao', 'do', 'dia', 'do', 'diplomata', '-', 'Palacio', 'Itamaraty', ',', '20', 'de', 'abril', 'de', '2018']
abril
33
3
['Discurso', 'do', 'Presidente', 'da', 'Republi

#### Verificando as datas:

In [46]:
print all_dates

[datetime.date(2018, 6, 18), datetime.date(2018, 5, 2), datetime.date(2018, 4, 20), datetime.date(2018, 4, 14), datetime.date(2018, 3, 19), datetime.date(2018, 3, 19), datetime.date(2018, 1, 24), datetime.date(2018, 1, 24), datetime.date(2017, 12, 21), datetime.date(2017, 12, 21), datetime.date(2017, 12, 21), datetime.date(2017, 12, 5), datetime.date(2017, 9, 20), datetime.date(2017, 9, 19), datetime.date(2017, 9, 5), datetime.date(2017, 9, 4), datetime.date(2017, 9, 4), datetime.date(2017, 9, 4), datetime.date(2017, 9, 3), datetime.date(2017, 9, 2), datetime.date(2017, 8, 21), datetime.date(2017, 8, 9), datetime.date(2017, 7, 21), datetime.date(2017, 7, 20), datetime.date(2017, 7, 7), datetime.date(2017, 7, 7), datetime.date(2017, 6, 29), datetime.date(2017, 6, 23), datetime.date(2017, 6, 23), datetime.date(2017, 6, 22), datetime.date(2017, 6, 21), datetime.date(2017, 6, 21), datetime.date(2017, 6, 20), datetime.date(2017, 6, 20), datetime.date(2017, 6, 5), datetime.date(2017, 5, 30),

#### Definindo a coluna de datas:

In [47]:
df_temer['dates'] = all_dates

#### O mesmo para Dilma:

In [48]:
all_titles_dilma = []
all_dates_dilma = []
for k in xrange(0,len(ftitle_dilma)):
    all_titles_dilma.append(unidecode(ftitle_dilma[k]))
    tokens_dilma = word_tokenize(all_titles_dilma[k])
    match_dilma = next((x for x in months if x in tokens_dilma), 'False')
    #print match_dilma, ' ', k
    if match_dilma != 'False':
        try:
            test_index_dilma = tokens_dilma.index(match_dilma)
            date_list_dilma = [months_dic[match_dilma], tokens_dilma[test_index_dilma-2],tokens_dilma[test_index_dilma+2]]
#            print tokens_dilma[test_index_dilma-2]
            if tokens_dilma[test_index_dilma-2] == '1deg' or tokens_dilma[test_index_dilma-2] == '1o':
                date_list_dilma[1] = '1'
#            print date_list_dilma
            fdate_dilma = str(' '.join(date_list_dilma))
#            datetime_object_dilma = datetime.strptime(fdate_dilma, '%b %d %Y')
            datetime_object_dilma = datetime.strptime(fdate_dilma, '%b %d %Y').date()
            all_dates_dilma.append(datetime_object_dilma)
              
        except:
            all_dates_dilma.append('not applicable')
            continue    
    else:
        print match_dilma
        all_dates_dilma.append('not applicable')

In [49]:
len(all_dates_dilma)

159

In [50]:
len(ftitle_dilma)

159

In [51]:
len(df_dilma['text'])

159

In [52]:
df_dilma['dates'] = all_dates_dilma

#### O mesmo para Lula:

In [53]:
all_titles_lula = []
all_dates_lula = []
for k in xrange(0,len(ftitle_lula)):
    all_titles_lula.append(unidecode(ftitle_lula[k]))
    tokens_lula = word_tokenize(all_titles_lula[k])
    match_lula = next((x for x in months if x in tokens_lula), 'False')
#    print match_lula, ' ', k
    if match_lula != 'False':
        try:
            test_index_lula = tokens_lula.index(match_lula)
            date_list_lula = [months_dic[match_lula], tokens_lula[test_index_lula-2],tokens_lula[test_index_lula+2]]
            if tokens_lula[test_index_lula-2] == '1deg' or tokens_lula[test_index_lula-2] == '1o':
                date_list_lula[1] = '1'
            fdate_lula = str(' '.join(date_list_lula))
#            datetime_object_lula = datetime.strptime(fdate_lula, '%b %d %Y')
            datetime_object_lula = datetime.strptime(fdate_lula, '%b %d %Y').date()
            all_dates_lula.append(datetime_object_lula)
              
        except:
            all_dates_lula.append('not applicable')
            continue    
    else:
        print match_lula
        all_dates_lula.append('not applicable')

False


In [54]:
len(all_dates_lula)

110

In [55]:
len(ftitle_lula)

110

In [56]:
df_lula['dates'] = all_dates_lula

#### Limpando os datasets (excluíndo as entradas com campo de texto nulo e com datas não determinadas):

In [57]:
df_lula_clean = df_lula[(df_lula['dates']!= 'not applicable') & (df_lula['text'] != '') ].reset_index(drop = True)

In [58]:
df_lula_clean

,title,text,president,dates
0,"Discurso do Presidente da República, Luiz Inác...","Excelentíssimo senhor Juan Manuel Santos, pres...",Lula,2010-09-01
1,"Discurso do Presidente da República, Luiz Iná...","Excelentíssimo senhor Malam Bacai Sanhá, presi...",Lula,2010-08-25
2,"Discurso do Presidente da República, Luiz Inác...","Se o Vinicius de Moraes estivesse aqui, ele ia...",Lula,2010-08-16
3,"Discurso do Presidente da República, Luiz Iná...","Meu caro companheiro e amigo Mauricio Funes, p...",Lula,2010-08-09
4,"Intervenção do Presidente da República, Luiz I...","Apenas, apenas para dizer aos companheiros Jos...",Lula,2010-08-06
5,"Discurso do Presidente da República, Luiz Iná...","Bem, companheiro Chávez,Companheiros empresári...",Lula,2010-08-06
6,"Discurso do Presidente da República, Luiz Inác...","Bem, primeiro, cumprimentar o nosso querido co...",Lula,2010-08-06
7,"Discurso do Presidente da República, Luiz Inác...","Excelentíssimo senhor Paul Biya, presidente do...",Lula,2010-08-04
8,"Discurso do Presidente da República, Luiz Inác...","Bem, primeiro, eu queria cumprimentar a compan...",Lula,2010-08-03
9,"Discurso do Presidente da República, Luiz Iná...","Eu sempre tenho uma dúvida, quando estou falan...",Lula,2010-07-30


In [59]:
df_dilma_clean = \
df_dilma[(df_dilma['dates']!= 'not applicable')  & (df_dilma['text'] != '')  ].reset_index(drop = True)

In [60]:
df_dilma_clean

,title,text,president,dates
0,"Discurso da Presidenta da República, Dilma Rou...",Foto: Roberto Stuckert Filho/PRSenhor Carlos A...,Dilma,2016-05-03
1,"Discurso da Presidenta da República, Dilma Rou...",Fotos: Ichiro Guerra/PRSenhor Secretário-Geral...,Dilma,2016-04-22
2,"Discurso da Presidenta da República, Dilma Rou...",Fotos: Ichiro Guerra/PRSenhor Secretário-Geral...,Dilma,2016-04-22
3,"Discurso da Presidenta da República, Dilma Rou...","Boa tarde a todos,Eu queria dirigir um cumprim...",Dilma,2016-02-26
4,"Pronunciamento da Presidenta da República, Dil...","Brasileiras e brasileiros,Peço licença para en...",Dilma,2016-02-03
5,"Discurso da Presidenta da República, Dilma Rou...","Excelentíssimo senhor Evo Morales, presidente ...",Dilma,2016-02-02
6,Declaração à imprensa da Presidenta da Repúbli...,Foto: Roberto Stuckert Filho/PR Excelentíssimo...,Dilma,2016-02-02
7,Declaração à imprensa da Presidenta da Repúbli...,Foto: Roberto Stuckert Filho/PR Excelentíssimo...,Dilma,2016-02-02
8,"Discurso da Presidenta da República, Dilma Rou...","Excelentíssimo senhor Rosen Plevneliev, presid...",Dilma,2016-02-01
9,Declaração à imprensa da Presidenta da Repúbli...,Foto: Roberto Stuckert/PR Excelentíssimo senho...,Dilma,2016-02-01


In [61]:
df_temer_clean =\
df_temer[(df_temer['dates']!= 'not applicable')  & (df_temer['text'] != '')  ].reset_index(drop = True)

#### Escrevendo os dataframes  em arquivos .csv e .xlsx (Excel):

In [62]:
writer = pd.ExcelWriter('temer.xlsx')
df_temer.to_excel(writer,'Sheet1',index=False)
writer.save()

writer = pd.ExcelWriter('temer_clean.xlsx')
df_temer_clean.to_excel(writer,'Sheet1',index=False)
writer.save()

df_temer.to_csv('temer.csv', sep ='\t', encoding='utf-8',index=False)
df_temer_clean.to_csv('temer_clean.csv', sep ='\t', encoding='utf-8',index=False)



writer = pd.ExcelWriter('dilma.xlsx')
df_dilma.to_excel(writer,'Sheet1',index=False)
writer.save()

writer = pd.ExcelWriter('dilma_clean.xlsx')
df_dilma_clean.to_excel(writer,'Sheet1',index=False)
writer.save()

df_dilma.to_csv('dilma.csv', sep ='\t', encoding='utf-8',index=False)
df_dilma_clean.to_csv('dilma_clean.csv', sep ='\t' , encoding='utf-8',index=False)



writer = pd.ExcelWriter('lula.xlsx')
df_lula.to_excel(writer,'Sheet1',index=False)
writer.save()

writer = pd.ExcelWriter('lula_clean.xlsx')
df_lula_clean.to_excel(writer,'Sheet1',index=False)
writer.save()

df_lula.to_csv('lula.csv', sep ='\t', encoding='utf-8',index=False)
df_lula_clean.to_csv('lula_clean.csv', sep ='\t', encoding='utf-8',index=False)

